In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import math
import glob
from sklearn import mixture
from sklearn.utils import shuffle
from skimage import measure
from glob import glob
import os
from scipy.misc import imsave


ROOT_DIR = 'data/cervix/'

DIR_TRAIN1 = ROOT_DIR + "train/Type_1"
DIR_TRAIN2 = ROOT_DIR + "train/Type_2"
DIR_TRAIN3 = ROOT_DIR + "train/Type_3"

DIR_TEST = ROOT_DIR + "test/test/"
DIR_ADDITIONAL_TYPE1 = ROOT_DIR + "additional_train/Type_1"
DIR_ADDITIONAL_TYPE2 = ROOT_DIR + "additional_train/Type_2"
DIR_ADDITIONAL_TYPE3 = ROOT_DIR + "additional_train/Type_3"

DIR_ROI1 = ROOT_DIR + "roi/Type_1/"
DIR_ROI2 = ROOT_DIR + "roi/Type_2/"
DIR_ROI3 = ROOT_DIR + "roi/Type_3/"

train_files1 = glob(os.path.join(DIR_TRAIN1, "*.jpg"))
train_files2 = glob(os.path.join(DIR_TRAIN2, "*.jpg"))
train_files3 = glob(os.path.join(DIR_TRAIN3, "*.jpg"))

test_files = glob(os.path.join(DIR_TEST, "*.jpg"))

additional_files_1 = glob(os.path.join(DIR_ADDITIONAL_TYPE1, "*.jpg"))
additional_files_2 = glob(os.path.join(DIR_ADDITIONAL_TYPE2, "*.jpg"))
additional_files_3 = glob(os.path.join(DIR_ADDITIONAL_TYPE3, "*.jpg"))

train_files1 = train_files1[:]
train_files2 = train_files2[:]
train_files3 = train_files3[:]
test_files = test_files[:3]
additional_files1 = additional_files_1[:]
additional_files2 = additional_files_2[:]
additional_files3 = additional_files_3[:]



def get_image_data(fname):
    img = cv2.imread(fname)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else:
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i - position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif (height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area = maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea


def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r - 1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (
    int(maxArea[3] + 1 - maxArea[0] / abs(maxArea[1] - maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])


def cropCircle(img):
    if (img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1] * 256 / img.shape[0]), 256)
    else:
        tile_size = (256, int(img.shape[0] * 256 / img.shape[1]))

    img = cv2.resize(img, dsize=tile_size)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    _, contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    ff = np.zeros((gray.shape[0], gray.shape[1]), 'uint8')
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0] + 2, gray.shape[1] + 2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1] / 2), int(gray.shape[0] / 2)), 1)
    # cv2.circle(ff, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 3, 3, -1)

    rect = maxRect(ff)
    img_crop = img[min(rect[0], rect[2]):max(rect[0], rect[2]), min(rect[1], rect[3]):max(rect[1], rect[3])]
    cv2.rectangle(ff, (min(rect[1], rect[3]), min(rect[0], rect[2])), (max(rect[1], rect[3]), max(rect[0], rect[2])), 3,
                  2)

    return img_crop


def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w * h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w / 2 - i) * (w / 2 - i) + (h / 2 - j) * (h / 2 - j))
            Ra[i * h + j, 0] = R
            Ra[i * h + j, 1] = min(imgLab[i][j][1], a_threshold)

    Ra[:, 0] /= max(Ra[:, 0])
    Ra[:, 0] *= Ra_ratio
    Ra[:, 1] /= max(Ra[:, 1])

    return Ra


def write_bbox_2_csv(csv2write, filespath, nfile_path, file_pre):
    out = open(csv2write, "w")
    out.write("image_name,type,clss,img_shp_0_init,img_shape1_init,img_shp_0,img_shp_1,sh0_start,sh1_start,sh0_end,sh1_end\n")
    for f in sorted(filespath):
        #print('Go for {}'.format(f))
        img_id = os.path.basename(f)
        img_type = 'test'
        clss = -1
        if DIR_TRAIN1 in f:
            img_type = 'train1'
            clss = int(os.path.basename(os.path.dirname(f)).split('_')[1]) - 1
        if DIR_TRAIN2 in f:
            img_type = 'train2'
            clss = int(os.path.basename(os.path.dirname(f)).split('_')[1]) - 1
        if DIR_TRAIN3 in f:
            img_type = 'train3'
            clss = int(os.path.basename(os.path.dirname(f)).split('_')[1]) - 1
        if DIR_ADDITIONAL_TYPE1 in f:
            img_type = 'add1'
            clss = int(os.path.basename(os.path.dirname(f)).split('_')[1]) - 1
        if DIR_ADDITIONAL_TYPE2 in f:
            img_type = 'add2'
            clss = int(os.path.basename(os.path.dirname(f)).split('_')[1]) - 1
        if DIR_ADDITIONAL_TYPE3 in f:
            img_type = 'add3'
            clss = int(os.path.basename(os.path.dirname(f)).split('_')[1]) - 1
        
        try:
            img = get_image_data(f)
            #img_ori = img # to be used for roi save
            #img_ori = cv2.imread(f)
        except:
            print('Image read error: {}. Skip it!'.format(f))
            continue
        if img is None:
            print('Problem with image: {}. Skip it!'.format(f))
            continue

        #print('image shape - ',img.shape)
        #print('image shape - original',img_ori.shape)
        #plt.subplot(131)
        #plt.title("Original image")    
        #plt.imshow(img) 

        initial_shape = img.shape[:2]

        img = cropCircle(img)
        w = img.shape[0]
        h = img.shape[1]

        imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)

        # Saturating the a-channel at 150 helps avoiding wrong segmentation
        # in the case of close-up cervix pictures where the bloody os is falsly segemented as the cervix.
        Ra = Ra_space(img, 1.0, 150)
        a_channel = np.reshape(Ra[:, 1], (w, h))
        #plt.subplot(121)
        #plt.imshow(a_channel)

        g = mixture.GaussianMixture(n_components=2, covariance_type='diag', random_state=0, init_params='kmeans')
        image_array_sample = shuffle(Ra, random_state=0)[:1000]
        g.fit(image_array_sample)
        labels = g.predict(Ra)
        labels += 1  # Add 1 to avoid labeling as 0 since regionprops ignores the 0-label.

        # The cluster that has the highest a-mean is selected.
        labels_2D = np.reshape(labels, (w, h))
        gg_labels_regions = measure.regionprops(labels_2D, intensity_image=a_channel)
        gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
        cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1

        mask = np.zeros((w * h, 1), 'uint8')
        mask[labels == cervix_cluster] = 255
        mask_2D = np.reshape(mask, (w, h))

        cc_labels = measure.label(mask_2D, background=0)
        regions = measure.regionprops(cc_labels)
        areas = [prop.area for prop in regions]

        regions_label = [prop.label for prop in regions]
        largestCC_label = regions_label[areas.index(max(areas))]
        mask_largestCC = np.zeros((w, h), 'uint8')
        mask_largestCC[cc_labels == largestCC_label] = 255

        img_masked = img.copy()
        img_masked[mask_largestCC == 0] = (0, 0, 0)
        img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY)

        _, thresh_mask = cv2.threshold(img_masked_gray, 0, 255, 0)

        kernel = np.ones((11, 11), np.uint8)
        thresh_mask = cv2.dilate(thresh_mask, kernel, iterations=1)
        thresh_mask = cv2.erode(thresh_mask, kernel, iterations=2)
        _, contours_mask, _ = cv2.findContours(thresh_mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        try:
            main_contour = sorted(contours_mask, key=cv2.contourArea, reverse=True)[0]
        except:
            print('Contour problem for {}. skip it!'.format(f))
            continue

        x, y, w, h = cv2.boundingRect(main_contour)
        roi_img = img[y:y+h, x:x+w]
        #roi = img_ori[y:y+h, x:x+w]
        #print('roi shape ',roi_img.shape)
        #print('roi shape for original',roi.shape)
        #get_ori_image_name = glob.glob(format(f))
        #get_ori_image_name = pd.DataFrame([[p.split('/')[7],p.split('/')[8],p] for p in get_ori_image_name], columns = ['type','image','path'])

        file_name = nfile_path+file_pre+img_id
        #print('roi file name ... ',file_name)
        #cv2.imwrite(file_name, roi_img)
        imsave(file_name, roi_img)
        
        #imageR = get_image_data(file_name)
        #dash='_'
        #for angle in np.arange(0, 90, 10):
        #    rotated = imutils.rotate(imageR, angle)
        #    file_nameR = nfile_path+file_pre+str(angle)+dash+img_id
        #    imsave(file_nameR, rotated)


        #cv2.rectangle(img,(x,y),(x+w,y+h),255,2)
        #plt.subplot(132)
        #plt.title("bbox")    
        #plt.imshow(img)

        #roi_img_load = get_image_data(file_name)
        #plt.subplot(133)
        #plt.title("roi")    
        #plt.imshow(roi_img_load)
        
        #plt.show()        
        
        
        out.write(img_id)
        out.write(',' + img_type)
        out.write(',' + str(clss))
        out.write(',' + str(initial_shape[0]))
        out.write(',' + str(initial_shape[1]))
        out.write(',' + str(img.shape[0]))
        out.write(',' + str(img.shape[1]))
        out.write(',' + str(y))
        out.write(',' + str(x))
        out.write(',' + str(y + h))
        out.write(',' + str(x + w))
        out.write('\n')
    
    out.close()

if __name__ == '__main__':

    print('DIR_ROI2 - additional_files2 ...')
    csv2write='cc_roi_type22a.csv'
    nfile_path=DIR_ROI2
    file_pre = 'roia_'
    write_bbox_2_csv(csv2write,additional_files2,nfile_path,file_pre)

    print('DIR_ROI2 - train_files2 ...')
    csv2write='cc_roi_type22.csv'
    nfile_path=DIR_ROI2
    file_pre = 'roi_'
    write_bbox_2_csv(csv2write,train_files2,nfile_path,file_pre)

    print('DIR_ROI1 - additional_files1 ...')
    csv2write='cc_roi_type11a.csv'
    nfile_path=DIR_ROI1
    file_pre = 'roia_'
    write_bbox_2_csv(csv2write,additional_files1,nfile_path,file_pre)

    print('DIR_ROI1 - train_files1 ...')
    csv2write='cc_roi_type11.csv'
    nfile_path=DIR_ROI1
    file_pre = 'roi_'
    write_bbox_2_csv(csv2write,train_files1,nfile_path,file_pre)

    print('DIR_ROI3 - additional_files3 ...')
    csv2write='cc_roi_type33a.csv'
    nfile_path=DIR_ROI3
    file_pre = 'roia_'
    write_bbox_2_csv(csv2write,additional_files3,nfile_path,file_pre)

    print('DIR_ROI3 - train_files3 ...')
    csv2write='cc_roi_type33.csv'
    nfile_path=DIR_ROI3
    file_pre = 'roi_'
    write_bbox_2_csv(csv2write,train_files3,nfile_path,file_pre)
    print('completed ...')

DIR_ROI2 - additional_files2 ...
DIR_ROI2 - train_files2 ...
DIR_ROI1 - additional_files1 ...
Contour problem for data/cervix/additional_train/Type_1/6361.jpg. skip it!
DIR_ROI1 - train_files1 ...
DIR_ROI3 - additional_files3 ...
DIR_ROI3 - train_files3 ...
completed ...


In [4]:
pwd

u'/home/cvpr/Documents/courses/deeplearning1/nbs'